# Task 4

In [1]:
import numpy as np
import pandas as pd
import csv
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.estimators import HillClimbSearch, BicScore, K2Score, MaximumLikelihoodEstimator

In [2]:
df = pd.read_csv('data/AND-Features.csv')
data = df.iloc[:, 2:]
data

,f1,f2,f3,f4,f5,f6,f7,f8,f9
0,2,1,1,0,2,2,0,2,2
1,2,1,1,0,2,2,0,1,2
2,2,1,1,3,2,2,0,2,2
3,1,1,1,0,2,2,0,2,2
4,2,1,1,0,2,2,0,0,2
5,2,1,1,3,2,2,0,1,2
6,2,1,1,0,2,2,0,3,2
7,1,1,1,0,2,2,0,1,2
8,1,1,1,3,2,2,0,2,2
9,2,1,1,3,2,2,0,0,2


## Bayesian network using HillClimbSearch

### BicScore

In [9]:
# BicScore
est = HillClimbSearch(data, scoring_method=BicScore(data))
best_model = est.estimate()
# search a model with restriction on the number of parents:
#est.estimate(max_indegree=1).edges()
print(sorted(best_model.nodes()))
print(sorted(best_model.edges()))
print(est.estimate(max_indegree=0).edges())

['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9']
[('f3', 'f5'), ('f4', 'f3'), ('f8', 'f9'), ('f9', 'f1'), ('f9', 'f5'), ('f9', 'f6'), ('f9', 'f7')]
[]


### Estimating MLE

In [10]:
model = BayesianModel([('f3', 'f5'), ('f4', 'f3'), ('f8', 'f9'), ('f9', 'f1'), ('f9', 'f5'), ('f9', 'f6'), ('f9', 'f7')])
estimator = MaximumLikelihoodEstimator(model, data)
df = pd.DataFrame(estimator.get_parameters())
print(df)

                                                   0
0  +-------+---------------------+---------------...
1  +-------+----------------------+--------------...
2  +-------+-----------+\n| f4(0) | 0.346004  |\n...
3  +-------+----------------------+--------------...
4  +-------+----------------------+--------------...
5  +-------+----------------------+--------------...
6  +-------+----------+\n| f8(0) | 0.170565 |\n+-...
7  +-------+---------------------+---------------...


## Estimating MLE

In [38]:
model = BayesianModel([('f3', 'f5'), ('f4', 'f3'), ('f8', 'f9'), ('f9', 'f1'), ('f9', 'f5'), ('f9', 'f6'), ('f9', 'f7')])
estimator = MaximumLikelihoodEstimator(model, data)
df = pd.DataFrame(estimator.get_parameters())
print(df)

for i in range(1, 10):
    try:
        cpd = MaximumLikelihoodEstimator(model, data).estimate_cpd('f' + str(i))
        print(cpd)
    except:
        print("Hello")

                                                   0
0  +-------+---------------------+---------------...
1  +-------+----------------------+--------------...
2  +-------+-----------+\n| f4(0) | 0.346004  |\n...
3  +-------+----------------------+--------------...
4  +-------+----------------------+--------------...
5  +-------+----------------------+--------------...
6  +-------+----------+\n| f8(0) | 0.170565 |\n+-...
7  +-------+---------------------+---------------...
+-------+---------------------+---------------------+
| f9    | f9(1)               | f9(2)               |
+-------+---------------------+---------------------+
| f1(0) | 0.19823788546255505 | 0.10638297872340426 |
+-------+---------------------+---------------------+
| f1(1) | 0.44933920704845814 | 0.26032540675844806 |
+-------+---------------------+---------------------+
| f1(2) | 0.2643171806167401  | 0.3692115143929912  |
+-------+---------------------+---------------------+
| f1(3) | 0.0881057268722467  | 0.264

In [11]:
K2Score(data).score(BayesianModel([('f3', 'f5'), ('f4', 'f3'), ('f8', 'f9'), ('f9', 'f1'), ('f9', 'f5'), ('f9', 'f6'), ('f9', 'f7')]))

-8175.27057163021

## Bayesian network using HillClimbSearch

In [54]:
# K2Score
est = HillClimbSearch(data, scoring_method=K2Score(data))
best_model = est.estimate()
print(sorted(best_model.nodes()))
print(sorted(best_model.edges()))
#est.estimate(max_indegree=9).edges()

['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9']
[('f3', 'f4'), ('f3', 'f8'), ('f3', 'f9'), ('f5', 'f3'), ('f5', 'f9'), ('f9', 'f1'), ('f9', 'f2'), ('f9', 'f4'), ('f9', 'f6'), ('f9', 'f7'), ('f9', 'f8')]


## Structure Score

In [55]:
K2Score(data).score(BayesianModel([('f3', 'f4'), ('f3', 'f8'), ('f3', 'f9'), ('f5', 'f3'), ('f5', 'f9'), ('f9', 'f1'), ('f9', 'f2'), ('f9', 'f4'), ('f9', 'f6'), ('f9', 'f7'), ('f9', 'f8')]))

-9462.704892371386

## Estimating MLE

In [56]:
model = BayesianModel([('f3', 'f4'), ('f3', 'f8'), ('f3', 'f9'), ('f5', 'f3'), ('f5', 'f9'), ('f9', 'f1'), ('f9', 'f2'), ('f9', 'f4'), ('f9', 'f6'), ('f9', 'f7'), ('f9', 'f8')])
estimator = MaximumLikelihoodEstimator(model, data)
df = pd.DataFrame(estimator.get_parameters())
print(df)

for i in range(1, 10):
    try:
        cpd = MaximumLikelihoodEstimator(model, data).estimate_cpd('f' + str(i))
        print(cpd)
    except:
        print("Hello")

                                                   0
0  +-------+---------------------+---------------...
1  +-------+----------------------+--------------...
2  +-------+---------------------+-------+-------...
3  +-------+---------------------+---------------...
4  +-------+-----------+\n| f5(0) | 0.178363  |\n...
5  +-------+----------------------+--------------...
6  +-------+----------------------+--------------...
7  +-------+---------------------+---------------...
8  +-------+---------------------+-------+-------...
+-------+---------------------+---------------------+
| f9    | f9(1)               | f9(2)               |
+-------+---------------------+---------------------+
| f1(0) | 0.19823788546255505 | 0.10638297872340426 |
+-------+---------------------+---------------------+
| f1(1) | 0.44933920704845814 | 0.26032540675844806 |
+-------+---------------------+---------------------+
| f1(2) | 0.2643171806167401  | 0.3692115143929912  |
+-------+---------------------+-------

In [57]:
len(model)

9

In [58]:
from pgmpy.models import BayesianModel
G = BayesianModel([('f3', 'f4'), ('f3', 'f8'), ('f3', 'f9'), ('f5', 'f3'), ('f5', 'f9'), ('f9', 'f1'), ('f9', 'f2'), ('f9', 'f4'), ('f9', 'f6'), ('f9', 'f7'), ('f9', 'f8')])
mm = G.to_markov_model()
print(mm.nodes())
print(mm.edges())


['f3', 'f4', 'f8', 'f9', 'f5', 'f1', 'f2', 'f6', 'f7']
[('f3', 'f4'), ('f3', 'f8'), ('f3', 'f9'), ('f3', 'f5'), ('f4', 'f9'), ('f8', 'f9'), ('f9', 'f1'), ('f9', 'f2'), ('f9', 'f6'), ('f9', 'f7'), ('f9', 'f5')]
